# Task1. 预测是酶还是非酶

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-08-26  

## 任务简介
该任务通过给定蛋白序列，预测该该蛋白是酶还是非酶。本任务所使用的数据集为Sport，对数据集的数据中进行学习，然后对新给定的蛋白序列数据预测是酶还是非酶。


## 数据统计
- 数据源Sprot，酶 219,227 条， 非酶226,539条。
- 将数据集中的所有数据按照时间排序，～90%作为训练集，～10%作为测试集，找到对应时间节点为2009年12月14日。
- 以2009年12月14日为时间节点，之前的数据为训练集，之后的数据为测试集，具体数据集统计如下： 

###  三级以下EC号的算作是酶 

|   Items   | 酶      |   非酶    |合计                               |
|-----------| --------| --------- |----------------------------------|
| 训练集     | 198,692 | 208,391   | 407,083（407,083/453,212=89.82%) |
| 测试集     | 20,535  | 25,594    | 46,129 (46,129/453,212=10.18% )  |


<div style="background-color: lightyellow; border-style: dotted; width:1100px; margin:auto; padding:35px;">


比对结果：
34813(查询有结果的)/46129（测试集总条数）

有查询结果：
酶: 18147/17919; 非酶: 16666/16894   
无查询结果的条数：11316
酶：2616； 非酶：8700
    
    
|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix						|
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |------------------------------------------|
|同源比对 <span style="color:red">「无比对结果的全算非酶」 </span> |0.915931    |	0.958946 	  |	 0.888035 | 0.847431|0.899747  | 	 	   |           |tp:17402  fp:745  fn:3133  tn:24849      |
|同源比对 <span style="color:red">「无比对结果的全算酶」 </span>   | 0.784040  |0.679428 |0.968979  |0.974823| 0.800752 | 	 |	| tp:20018  fp:9445  fn:517  tn:16149|
|同源比对 <span style="color:red">「无比对结果的随机猜测1/2算酶，1/2非酶」 </span>   | 0.848685  |0.784351| 0.917467  |0.910397|0.842686| 	| 	 	| tp:18695  fp:5140  fn:1840  tn:20454|
|同源比对 <span style="color:red">「只计算有比对结果的」 </span>	|0.963749  	|0.958946 		   |0.968979 | 	0.971148 |0.965009	|	 	 	| 	| tp:17402  fp:745  fn:517  tn:16149|
|DeepEC	 	| 0.848013  |	0.979370 	|	 0.790151| 0.672754|0.797610  |		 	 |           |tp:13815  fp:291  fn:6720  tn:25303|
|集成XGBOOST | 0.935615  |	0.919272 |	 0.949210|  0.937716	 |0.928403		 |	 |	 |	 tp:19256  fp:1691  fn:1279  tn:23903|
|集成dt 	 |0.912658  	|0.881448 	| 0.940228  |	0.928707	 |0.904460		 |	 |	 | tp:19071  fp:2565  fn:1464  tn:23029|
    
</div>

</br></br>

###  三级以下EC号的扔掉，不包括到数据集中 
    
|    Items    | 酶       |   非酶    |合计                               |
| ------------ | --------| --------- |----------------------------------|
| 训练集        | 198,692 | 202,602   | 401,294（401,294/445,798=90.02%)  |
| 测试集        | 20,535  | 23,969    | 44,504 (44,504/445,798= 9.98% )  |


<div style="background-color: lightgreen; border-style: dotted; width:1100px; margin:auto; padding:35px;">
    

比对结果：
33514(查询有结果的)/44504（测试集总条数）

有查询结果：
酶: 17551/17854; 非酶: 15344/15660   
无查询结果的条数：10990
酶：2681； 非酶：10990
    
    
    
|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix						|
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |------------------------------------------|
|同源比对 <span style="color:red">「无比对结果的全算非酶」 </span>  |0.925849  	|0.982314 	|0.887975  |0.854687 | 0.914067	| 	| 	| 	 tp:17551  fp:316  fn:2984  tn:23653|
|同源比对 <span style="color:red">「无比对结果的全算酶」 </span>   | 0.799389 |0.701112   | 0.980635  | 0.985245	| 0.819242	|	 |	| 	  tp:20232  fp:8625  fn:303  tn:15344|
|同源比对 <span style="color:red">「无比对结果的随机猜测1/2算酶，1/2非酶」 </span>   | 0.864394|  	0.810599| 	0.923731|  	0.921402|	 0.862456|	|	| tp:18921  fp:4421  fn:1614  tn:19548|
|同源比对 <span style="color:red">「只计算有比对结果的」 </span>	|0.981530  	|0.982314|0.980635 | 	0.983029	| 0.982671|     |		| 	 tp:17551  fp:316  fn:303  tn:15344|
|同源比对<span style="color:red">「无比对结果的算错误」 </span>  	| 0.739147  |0.751263| 0.725759|  	0.751681	| 0.751472|		 |	 	| 	 tp:17551  fp:5811  fn:5798  tn:15344|
|DeepEC	 	 | 0.842913  |	0.980761  |0.779078  |	0.672754 | 0.798071	| 	 |	 |	 tp:13815  fp:271  fn:6720  tn:23698|
|ECPred	 	 |0.898593   |0.836455 	  | 0.970088 |0.969856	 |0.898230	|	 |	 |	 tp:19916  fp:3894  fn:619  tn:20075|
|集成lr 	 	|0.948454  	|0.941863 	|0.954155  	|0.946725	 |0.944288	|	| 	| 	 tp:19441  fp:1200  fn:1094  tn:22769|
|集成xg 	 	|0.952431  	|0.947691 	|0.956505  	|0.949306	 |0.948498	|	| 	| 	 tp:19494  fp:1076  fn:1041  tn:22893|
|集成dt 	 	|0.926816  	|0.904827 	|0.947073  	|0.940297	 |0.922221	|	| 	| 	 tp:19309  fp:2031  fn:1226  tn:21938|
|集成rf 	 	|0.952746  	|0.954214 	|0.951516  	|0.942829	 |0.948487	|	| 	| 	 tp:19361  fp:929   fn:1174  tn:23040|
|集成gbdt 	|0.951488  	|0.949863 	|0.952865  	|0.944729	 |0.947289	|	| 	| 	 tp:19400  fp:1024  fn:1135  tn:2294 |   
</div>









## 实验方法

- 同源比对：使用训练集建立比对库，然后将测试集与之比对，取最优的比对结果，比对结果的（酶/非酶）属性当作测试集的测试结果
- 传统机器学习方法
- 深度学习方法



## 实验结果

### baselines




<div style="background-color: lightyellow; border-style: dotted; width:1100px; margin:auto; padding:35px;">

###  三级以下EC号的算作是酶     
    
### 机器学习 + onehot


|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix						|
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |------------------------------------------|
| lr 		|	0.609676 |	0.560377 	 |	0.680254 |0.715023 |0.628324 |	0.661484 |	0.568861 |	 tp: 14683 fp: 11519 fn: 5852 tn: 12450	|
| xg 		|	0.665446 |	0.643591 	 |	0.682740 |0.616168 |0.629581 |	0.742674 |	0.684483 |	 tp: 12653 fp: 7007 fn: 7882 tn: 16962	|
| dt 		|	0.602418 |	0.570965 	 |	0.628129 |0.556562 |0.563671 |	0.599133 |	0.522388 |	 tp: 11429 fp: 8588 fn: 9106 tn: 15381	|
| rf 		|	0.666479 |	0.624934 	 |	0.710044 |0.693255 |0.657324 |	0.745132 |	0.689971 |	 tp: 14236 fp: 8544 fn: 6299 tn: 15425	|
| gbdt  	| 	0.624079 |	0.569351 	 |	0.712026 |0.760604 |0.651226 |	0.695638 |	0.628716 |	 tp: 15619 fp: 11814 fn: 4916 tn: 12155	|


### 机器学习 + unirep

|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix					  |
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |----------------------------------------|
|lr 	    |0.826757 	 |	0.849132 	 |	0.811034 |0.759484 |0.801810 |	0.901500 |	0.892720 |	 tp: 15596 fp: 2771 fn: 4939 tn: 21198|
|xg 	    |0.860260	 |	0.883725 	 |	0.843327 |0.802776 |0.841308 |	0.934751 |	0.927646 |	 tp: 16485 fp: 2169 fn: 4050 tn: 21800|
|dt 	    |0.772290 	 |	0.767612 	 |	0.775916 |0.726418 |0.746447 |	0.769004 |	0.683843 |	 tp: 14917 fp: 4516 fn: 5618 tn: 19453|
|rf 	    |0.851070    |	0.903031 	 |	0.818172 |0.758705 |0.824600 |	0.932207 |	0.922548 |	 tp: 15580 fp: 1673 fn: 4955 tn: 22296|
|gbdt 		|0.822825 	 |0.864764 		 |0.796054 	 |0.730217 |0.791815 |	0.898601 |	0.889249 |	 tp: 14995 fp: 2345 fn: 5540 tn: 21624|

</div>

</br>


<div style="background-color: lightgreen; border-style: dotted; width:1100px; margin:auto; padding:35px;">

###  三级以下EC号的扔掉，不包括到数据集中     
    
### 机器学习 + onehot


|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix						|
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |------------------------------------------|
|lr 		|0.608664 |0.560196 |0.676184 |	0.706745 |	0.624995 |	0.658145 |	0.565398 	| tp: 14513 fp: 11394 fn: 6022 tn: 12575	|
|xg 		|0.671468 |0.649797 |0.688742 |	0.624641 |	0.636971 |	0.744063 |	0.685988 	| tp: 12827 fp: 6913 fn: 7708 tn: 17056		|
|dt 		|0.606260 |0.574821 |0.632220 |	0.563428 |	0.569068 |	0.603192 |	0.525313 	| tp: 11570 fp: 8558 fn: 8965 tn: 15411		|
|rf 		|0.664435 |0.622416 |0.708882 |	0.693402 |	0.655994 |	0.744528 |	0.689728 	| tp: 14239 fp: 8638 fn: 6296 tn: 15331		|
|gbdt 		|0.622910 |0.568463 |0.710208 |	0.758753 |	0.649967 |	0.694911 |	0.631114 	| tp: 15581 fp: 11828 fn: 4954 tn: 12141	|


### 机器学习 + unirep

|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix					  |
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |----------------------------------------|
|lr 	|0.825521 	|0.847237 	|0.810231 |	0.758656 |	0.800504 |	0.900743 |	0.891768 |	 tp: 15579 fp: 2809 fn: 4956 tn: 21160|
|xg 	|0.857788 	|0.881841 	|0.840514 |	0.798831 |	0.838286 |	0.933706 |	0.927064 |	 tp: 16404 fp: 2198 fn: 4131 tn: 21771|
|dt 	|0.771346 	|0.766943 	|0.774750 |	0.724665 |	0.745205 |	0.768002 |	0.682822 |	 tp: 14881 fp: 4522 fn: 5654 tn: 19447|
|rf 	|0.849811 	|0.901548 	|0.817075 |	0.757195 |	0.823090 |	0.931437 |	0.922190 |	 tp: 15549 fp: 1698 fn: 4986 tn: 22271|

</div>

## 1. 导入必要的包、定义公共函数

In [1]:
import numpy as np
import pandas as pd
import random
import time
import datetime
import sys
import os
from tqdm import tqdm
from functools import reduce

sys.path.append("../../tools/")
import commontools
import funclib
import ucTools

sys.path.append("../../")
import benchmark_common as bcommon
import benchmark_train as btrain
import benchmark_test as btest
import benchmark_config as cfg
import benchmark_evaluation as eva

from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize() # 初始化该这个b...并行库

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. 加载数据

In [2]:
train = pd.read_feather(cfg.DATADIR+'train.feather')
test = pd.read_feather(cfg.DATADIR+'test.feather')

print('train size: {0}\ntest size: {1}'.format(len(train), len(test)))

trainset = train[['id', 'isemzyme','seq', 'seqlength']].reset_index(drop=True)
testset = test[['id', 'isemzyme','seq', 'seqlength']].reset_index(drop=True)

train size: 469129
test size: 8033


In [3]:
train.head(3)

,id,isemzyme,functionCounts,ec_number,ec_specific_level,seq,seqlength,f1,f2,f3,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,A0A023GPI8,False,0,-,0,ADTIVAVELDTYPNTDIGDPSYPHIGIDIKSVRSKKTAKWNMQNGK...,236,0.000115,0.105367,0.037413,...,0.052682,-0.020702,0.081852,0.008791,-0.036727,-0.027552,-0.254128,0.020778,0.140904,0.140635
1,A0A023GPJ0,False,0,-,0,MFGIFSKGEPVSMEGELVQPSSIVINDYEEELHLPLSYWDIKDYKN...,143,0.001688,-0.017209,0.029518,...,0.040570,-0.031992,-0.174783,0.000952,-0.136715,0.042773,0.132279,0.015899,0.152930,-0.003412
2,A0A023PXA5,False,0,-,0,MLLSELVATASSLPYTAISIHNNCRVPAARHIHHGCRYFHGPPVMH...,189,0.003066,-0.010453,0.013398,...,0.014996,0.034800,-0.221564,0.014209,-0.019194,-0.006493,0.121753,0.040914,0.143036,-0.024102


## 3. 同源比对

In [8]:
# 读取比对结果
res_data=funclib.getblast_fasta(trainfasta=cfg.TRAIN_FASTA,testfasta=cfg.TEST_FASTA)
print(' query samples:{0}\n results samples: {1}'.format(len(test), len(res_data)))

diamond makedb --in /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1
 query samples:8033
 results samples: 6034


In [9]:
# 给比对结果添加标签
isEmzyme_dict = {v: k for k,v in zip(train.isemzyme, train.id )} 
res_data['isEmzyme_pred'] = res_data['sseqid'].apply(lambda x: isEmzyme_dict.get(x))

isEmzyme_dict_test = {v: k for k,v in zip(test.isemzyme, test.id )} 
res_data['isEmzyme_groudtruth'] = res_data['id'].apply(lambda x: isEmzyme_dict_test.get(x))

In [10]:

# 获取比对结果
res_data_sub = res_data[['id','isEmzyme_pred']]

#选取测试集的ID与是否是酶2列
test_data_sub=test[['id', 'isemzyme']]
test_data_sub.reset_index(drop=True, inplace=True)

#拼合比对结果到测试集
res = pd.merge(test_data_sub,res_data_sub, on='id', how='left')

# 填充没有比对结果的值为非酶
res.fillna(False, inplace=True)

# # # 填充没有比对结果的值为酶
# res.fillna(True, inplace=True)

# 填充没有比对结果的值 随机填充
# list_enornot=[True, False]
# for i in range(len(res[(res.isEmzyme_pred.isnull())])):
#     res.fillna(random.choice(list_enornot), inplace=True, limit=1)


In [11]:
# 计算指标
tp = len(res[res.isemzyme & res.isEmzyme_pred])
fp = len(res[(res.isemzyme ==False) & (res.isEmzyme_pred)])
tn = len(res[(res.isemzyme ==False) & (res.isEmzyme_pred ==False)])
fn = len(res[(res.isemzyme ) & (res.isEmzyme_pred == False)])
funclib.caculateMetrix_1('同源比对',tp, fp, tn,fn)

同源比对 	 0.902776  	0.969148 		 0.863591  	0.807488	 0.880963		 	 	 	 tp:2890  fp:92  fn:689  tn:4362


## 4. 机器学习方法预测
### 4.1 one-hot + 机器学习

In [126]:
MAX_SEQ_LENGTH = 1500 #定义序列最长的长度
trainset.seq = trainset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))
testset.seq = testset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))

In [131]:
f_train = funclib.dna_onehot(trainset) #训练集编码
f_test = funclib.dna_onehot(testset) #测试集编码

In [139]:
# 计算指标
X_train = np.array(f_train.iloc[:,2:])
X_test = np.array(f_test.iloc[:,2:])
Y_train = np.array(f_train.isemzyme.astype('int'))
Y_test = np.array(f_test.isemzyme.astype('int'))
funclib.run_baseline(X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 		0.647330 	0.588599 		0.712006 	0.692372 	0.636282 	0.710190 	0.591010 	 tp: 2478 fp: 1732 fn: 1101 tn: 2722
xg 		0.708702 	0.685091 		0.725566 	0.640682 	0.662143 	0.792299 	0.709043 	 tp: 2293 fp: 1054 fn: 1286 tn: 3400
dt 		0.628283 	0.588906 		0.656237 	0.548757 	0.568123 	0.620472 	0.524212 	 tp: 1964 fp: 1371 fn: 1615 tn: 3083
rf 		0.716171 	0.676831 		0.749312 	0.694607 	0.685604 	0.807917 	0.742798 	 tp: 2486 fp: 1187 fn: 1093 tn: 3267
gbdt 		0.677455 	0.616290 		0.746103 	0.731489 	0.668966 	0.748616 	0.646849 	 tp: 2618 fp: 1630 fn: 961 tn: 2824


### 5. Unirep + 机器学习

In [152]:
X_train = np.array(train.iloc[:,7:])
X_test = np.array(test.iloc[:,7:])

Y_train = np.array(train.isemzyme.astype('int')).flatten()
Y_test = np.array(test.isemzyme.astype('int')).flatten()

funclib.run_baseline(X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 		0.836798 	0.859316 		0.822227 	0.757754 	0.805345 	0.915379 	0.897222 	 tp: 2712 fp: 444 fn: 867 tn: 4010
xg 		0.865057 	0.888508 		0.849440 	0.797150 	0.840353 	0.940442 	0.928549 	 tp: 2853 fp: 358 fn: 726 tn: 4096
dt 		0.796589 	0.795144 		0.797594 	0.732048 	0.762293 	0.790249 	0.701466 	 tp: 2620 fp: 675 fn: 959 tn: 3779
rf 		0.863438 	0.915050 		0.832838 	0.764459 	0.833004 	0.943371 	0.930049 	 tp: 2736 fp: 254 fn: 843 tn: 4200
gbdt 		0.833188 	0.873790 		0.809051 	0.731210 	0.796167 	0.916202 	0.899225 	 tp: 2617 fp: 378 fn: 962 tn: 4076


### 6. ESM + 机器学习
#### 6.1 REP33

In [4]:
train_esm_33 = pd.read_feather(cfg.DATADIR + 'train_rep33.feather')
test_esm_33 = pd.read_feather(cfg.DATADIR + 'test_rep33.feather')

train_esm = trainset.merge(train_esm_33, on='id', how='left')
test_esm = testset.merge(test_esm_33, on='id', how='left')

In [5]:
X_train = np.array(train_esm.iloc[:,4:])
X_test = np.array(test_esm.iloc[:,4:])

Y_train = np.array(train_esm.isemzyme.astype('int')).flatten()
Y_test = np.array(test_esm.isemzyme.astype('int')).flatten()

funclib.run_baseline(X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 		0.903150 	0.920444 		0.890897 	0.856664 	0.887410 	0.959831 	0.946440 	 tp: 3066 fp: 265 fn: 513 tn: 4189
xg 		0.915723 	0.944274 		0.896161 	0.861693 	0.901096 	0.973722 	0.966651 	 tp: 3084 fp: 182 fn: 495 tn: 4272
dt 		0.831072 	0.844176 		0.822268 	0.761386 	0.800646 	0.824227 	0.749055 	 tp: 2725 fp: 503 fn: 854 tn: 3951
rf 		0.916096 	0.960965 		0.887136 	0.846046 	0.899851 	0.973453 	0.968153 	 tp: 3028 fp: 123 fn: 551 tn: 4331
gbdt 		0.865804 	0.901703 		0.843089 	0.784297 	0.838912 	0.942312 	0.926391 	 tp: 2807 fp: 306 fn: 772 tn: 4148


#### 6.2 REP32

In [4]:
train_esm_32 = pd.read_feather(cfg.DATADIR + 'train_rep32.feather')
test_esm_32 = pd.read_feather(cfg.DATADIR + 'test_rep32.feather')

train_esm = trainset.merge(train_esm_32, on='id', how='left')
test_esm = testset.merge(test_esm_32, on='id', how='left')

X_train = np.array(train_esm.iloc[:,4:])
X_test = np.array(test_esm.iloc[:,4:])

Y_train = np.array(train_esm.isemzyme.astype('int')).flatten()
Y_test = np.array(test_esm.isemzyme.astype('int')).flatten()

funclib.run_baseline(X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 		0.906635 	0.920857 		0.896404 	0.864767 	0.891931 	 tp: 3095 fp: 266 fn: 484 tn: 4188
[0]	validation_0-logloss:0.57080	validation_1-logloss:0.57030
[1]	validation_0-logloss:0.49022	validation_1-logloss:0.48923
[2]	validation_0-logloss:0.43134	validation_1-logloss:0.43065
[3]	validation_0-logloss:0.39128	validation_1-logloss:0.39051
[4]	validation_0-logloss:0.35889	validation_1-logloss:0.35843
[5]	validation_0-logloss:0.32997	validation_1-logloss:0.33010
[6]	validation_0-logloss:0.30737	validation_1-logloss:0.30785
[7]	validation_0-logloss:0.28818	validation_1-logloss:0.28919
[8]	validation_0-logloss:0.27361	validation_1-logloss:0.27513
[9]	validation_0-logloss:0.25849	validation_1-logloss:0.26025
[10]	validation_0-logloss:0.24510	validation_1-logloss:0.24707
[11]	validation_0-logloss:0.23391	validation_1-logloss:0.23622
[12]	validation_0-logloss:0.22314	validation_1-logloss:0.22573
[

#### 6.3 REP0

In [5]:
train_esm_0 = pd.read_feather(cfg.DATADIR + 'train_rep0.feather')
test_esm_0 = pd.read_feather(cfg.DATADIR + 'test_rep0.feather')

train_esm = trainset.merge(train_esm_0, on='id', how='left')
test_esm = testset.merge(test_esm_0, on='id', how='left')

X_train = np.array(train_esm.iloc[:,4:])
X_test = np.array(test_esm.iloc[:,4:])

Y_train = np.array(train_esm.isemzyme.astype('int')).flatten()
Y_test = np.array(test_esm.isemzyme.astype('int')).flatten()

funclib.run_baseline(X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 		0.756131 	0.720949 		0.785665 	0.738474 	0.729607 	 tp: 2643 fp: 1023 fn: 936 tn: 3431
[0]	validation_0-logloss:0.59910	validation_1-logloss:0.59933
[1]	validation_0-logloss:0.54513	validation_1-logloss:0.54538
[2]	validation_0-logloss:0.50851	validation_1-logloss:0.50933
[3]	validation_0-logloss:0.48410	validation_1-logloss:0.48527
[4]	validation_0-logloss:0.46540	validation_1-logloss:0.46730
[5]	validation_0-logloss:0.45156	validation_1-logloss:0.45379
[6]	validation_0-logloss:0.44073	validation_1-logloss:0.44330
[7]	validation_0-logloss:0.43070	validation_1-logloss:0.43372
[8]	validation_0-logloss:0.42179	validation_1-logloss:0.42518
[9]	validation_0-logloss:0.41608	validation_1-logloss:0.41990
[10]	validation_0-logloss:0.41114	validation_1-logloss:0.41531
[11]	validation_0-logloss:0.40489	validation_1-logloss:0.40948
[12]	validation_0-logloss:0.40045	validation_1-logloss:0.40570


In [17]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [ ]:
def train_howmany_enzyme(data_x, data_y, vali_ratio=0.3, force_model_update=False):
    x_train, x_vali, y_train, y_vali = train_test_split(data_x,np.array(data_y).ravel(),test_size=vali_ratio,random_state=1)
    eval_set = [(x_train, y_train), (x_vali, y_vali)]

    model = XGBClassifier(
                            min_child_weight=1, 
                            max_depth=6, 
                            objective='multi:softmax', 
                            num_class=2, 
                            use_label_encoder=False,
                            n_estimators=200,
                            n_jobs=0,
                            gamma=0.003
                        )
    print("-" * 100)
    print("几功能酶xgboost模型：", model)
    model.fit(x_train, y_train, eval_metric="mlogloss", eval_set=eval_set, verbose=True)
    # 保存模型

    return model

In [9]:
train_esm_32 = pd.read_feather(cfg.DATADIR + 'train_rep32.feather')
test_esm_32 = pd.read_feather(cfg.DATADIR + 'test_rep32.feather')

train_esm = trainset.merge(train_esm_32, on='id', how='left')
test_esm = testset.merge(test_esm_32, on='id', how='left')

X_train = np.array(train_esm.iloc[:,4:])
X_test = np.array(test_esm.iloc[:,4:])

Y_train = np.array(train_esm.isemzyme.astype('int')).flatten()
Y_test = np.array(test_esm.isemzyme.astype('int')).flatten()

In [ ]:
model_xg=train_howmany_enzyme(data_x=X_train, data_y=Y_train.flatten(),  vali_ratio=0.3, force_model_update=False)

----------------------------------------------------------------------------------------------------
几功能酶xgboost模型： XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=0, num_class=2, num_parallel_tree=None,
              objective='multi:softmax', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, use_label_encoder=False,
              validate_parameters=None, verbosity=None)
[0]	validation_0-mlogloss:0.56958	validation_1-mlogloss:0.56948
[1]	validation_0-mlogloss:0.49130	validation_1-mlogloss:0.49154
[2]	validation_0-mlogloss:0.43331	validation_1-mloglos

In [41]:
predict = model_xg.predict(X_test)
predictprob = model_xg.predict_proba(X_test)
eva.caculateMetrix(groundtruth=test.isemzyme, predict=predict, baselineName='xgboost', type='binary')

xgboost 		0.925433 	0.953991 		0.905704 	0.874825 	0.912695 	 tp: 3131 fp: 151 fn: 448 tn: 4303


In [61]:
predict = model_xg.predict(X_test)
predictprob = model_xg.predict_proba(X_test)
eva.caculateMetrix(groundtruth=test.isemzyme, predict=predict, baselineName='xgboost', type='binary')

xgboost 		0.925557 	0.949081 		0.908994 	0.880134 	0.913308 	 tp: 3150 fp: 169 fn: 429 tn: 4285
